### ✅ Task 4: Alternative Pathfinding Algorithm – Dijkstra vs A*

In this task, we implement and compare:
- **Dijkstra's Algorithm** (with constraint penalties)
- **A\*** using:
  - Euclidean heuristic
  - Constraint-aware heuristic

We also count **nodes expanded** to evaluate efficiency.


In [ ]:

import networkx as nx
import math
import heapq

# Define points and segments
points = {
    'A': (-37.8156505, 145.2294636), 'B': (-37.8154740, 145.2294588), 'C': (-37.8152886, 145.2288074),
    'D': (-37.8148201, 145.2290135), 'E': (-37.8149849, 145.2290695), 'F': (-37.8140786, 145.2296805),
    'G': (-37.8144825, 145.2296467), 'H': (-37.8139861, 145.2306309), 'I': (-37.8144433, 145.2300744),
    'J': (-37.8136444, 145.2314192), 'K': (-37.8146830, 145.2292295), 'L': (-37.8140991, 145.2303492),
    'M': (-37.8148709, 145.2295034)
}

segments = [
    ('A', 'B', 20), ('B', 'C', 65), ('C', 'D', 55), ('D', 'E', 19), ('E', 'F', 80),
    ('F', 'G', 45), ('G', 'H', 95), ('H', 'I', 70), ('I', 'J', 110), ('J', 'K', 150),
    ('K', 'L', 90), ('L', 'M', 65), ('M', 'A', 128), ('D', 'K', 43), ('C', 'G', 43),
    ('E', 'G', 57), ('G', 'I', 61), ('F', 'I', 88), ('K', 'I', 78), ('H', 'J', 143),
    ('A', 'D', 72), ('B', 'E', 78), ('C', 'E', 60), ('D', 'F', 84), ('G', 'J', 135),
    ('H', 'K', 122), ('I', 'M', 75), ('J', 'L', 80), ('F', 'L', 105), ('A', 'G', 90)
]

constraints = [
    'none', 'slope', 'none', 'obstacle', 'slope', 'kerb_ramp', 'obstacle', 'slope', 'kerb_ramp', 'none',
    'slope', 'kerb_ramp', 'none', 'none', 'slope', 'obstacle', 'kerb_ramp', 'slope', 'none', 'none',
    'obstacle', 'kerb_ramp', 'slope', 'none', 'obstacle', 'none', 'kerb_ramp', 'slope', 'none', 'kerb_ramp'
]

# Create graph with constraints
G_task3 = nx.Graph()
for point, (lat, lon) in points.items():
    G_task3.add_node(point, pos=(lon, lat))

for (u, v, dist), constraint in zip(segments, constraints):
    G_task3.add_edge(u, v, weight=dist, constraint=constraint)

# Define constraint penalty
constraint_penalty = {
    'none': 0,
    'kerb_ramp': 2,
    'slope': 5,
    'obstacle': 1000
}

# Heuristics
def euclidean_heuristic(a, b):
    ax, ay = points[a][1], points[a][0]
    bx, by = points[b][1], points[b][0]
    return math.hypot(bx - ax, by - ay) * 100000

def constraint_aware_heuristic(current, goal):
    base = euclidean_heuristic(current, goal)
    penalties = [
        constraint_penalty.get(G_task3[current][neighbor]['constraint'], 0)
        for neighbor in G_task3.neighbors(current)
    ]
    avg_penalty = sum(penalties) / len(penalties) if penalties else 0
    return base + avg_penalty

# A* with expansion tracking
def a_star_with_expansion(graph, start, goal, heuristic_func):
    open_set = [(heuristic_func(start, goal), 0, start, [])]
    visited = set()
    expanded = 0
    while open_set:
        f, g, current, path = open_set.pop(0)
        if current in visited:
            continue
        visited.add(current)
        expanded += 1
        path = path + [current]
        if current == goal:
            return path, g, expanded
        for neighbor in graph.neighbors(current):
            weight = graph[current][neighbor]['weight']
            constraint = graph[current][neighbor]['constraint']
            cost = weight + constraint_penalty[constraint]
            if neighbor not in visited:
                new_g = g + cost
                new_f = new_g + heuristic_func(neighbor, goal)
                open_set.append((new_f, new_g, neighbor, path))
        open_set.sort()
    return None, float('inf'), expanded

# Dijkstra with expansion tracking
def dijkstra_with_expansion(graph, start, goal):
    queue = [(0, start, [])]
    visited = set()
    expanded = 0
    while queue:
        cost, current, path = heapq.heappop(queue)
        if current in visited:
            continue
        visited.add(current)
        expanded += 1
        path = path + [current]
        if current == goal:
            return path, cost, expanded
        for neighbor in graph.neighbors(current):
            if neighbor not in visited:
                edge_weight = graph[current][neighbor]['weight']
                constraint = graph[current][neighbor]['constraint']
                penalty = constraint_penalty[constraint]
                total_cost = cost + edge_weight + penalty
                heapq.heappush(queue, (total_cost, neighbor, path))
    return None, float('inf'), expanded

# Run all algorithms
dj_path, dj_cost, dj_exp = dijkstra_with_expansion(G_task3, 'A', 'J')
as_path, as_cost, as_exp = a_star_with_expansion(G_task3, 'A', 'J', euclidean_heuristic)
ca_path, ca_cost, ca_exp = a_star_with_expansion(G_task3, 'A', 'J', constraint_aware_heuristic)

# Output results
print("🔍 Pathfinding Comparison from A to J\n")
print(f"Dijkstra:              {dj_path} | Cost: {round(dj_cost, 2)} | Nodes Expanded: {dj_exp}")
print(f"A* (Euclidean):        {as_path} | Cost: {round(as_cost, 2)} | Nodes Expanded: {as_exp}")
print(f"A* (Constraint-Aware): {ca_path} | Cost: {round(ca_cost, 2)} | Nodes Expanded: {ca_exp}")
